# Tema 9: expresiones regulares (III)
Vamos a ver los conceptos más avanzados de las expresiones regulares.

## Avaricia y pereza
Los cuantificadores que hemos visto en el cuaderno anterior, `?`, `+` y `*` son, por defecto, avariciosos (_greedy_); eso quiere decir que abarcarán todo lo que puedan.

Las etiquetas HTML son ejemplos típicos. Imaginemos que tenemos un texto con etiquetas HTML y queremos quitárselas todas:

In [2]:
import re

HTML = "Podemos llamarlas <em>expresiones regulares</em>, <em>regexp</em> o <em>regex</em>."

La regex `<.+>` a priori es muy suculenta para cazar cada etiqueta, pero en vez de capturar cada una de las etiquetas, lo que hace es capturar desde el primer `<` hasta el último `>`, porque es avariciosa:

In [37]:
a = re.findall("<.+>", HTML)
print(a)

['<em>expresiones regulares</em>, <em>regexp</em> o <em>regex</em>']


Tampoco nos sirve restringirlo a fragmentos de texto sin espacios, porque hay fragmentos de texto entre etiqueta y etiqueta que pueden ser una palabra sola:

In [38]:
a = re.findall("<[^ ]+>", HTML)
print(a)

['<em>', '</em>', '<em>regexp</em>', '<em>regex</em>']


Solución: con `?` la hacemos perezosa (_lazy_); es decir, dejará de buscar tras la primera instancia de `>`. Con `<.+?>` capturamos todas las etiquetas y solo las etiquetas.

In [39]:
a = re.findall("<.+?>", HTML)
print(a)

['<em>', '</em>', '<em>', '</em>', '<em>', '</em>']


Ahora ya podemos usar `re.sub()` para sustituirlas por una cadena vacía y obtener el texto limpio:

In [3]:
a = re.sub("<.+?>", "", HTML)
print(a)

Podemos llamarlas expresiones regulares, regexp o regex.


Para distinguir entre este `?` y el cuantificador `?`, piensa que cada uno es distinto; el que acabamos de aprender va necesariamente detrás de un cuantificador indeterminado (`?`, `+` o `*`, vaya). El otro es un cuantificador y siempre va detrás de un carácter (ya sea literal o metacarácter, expresado como un rango, como un conjunto, como un grupo (que ahora aprenderemos), etc.).

## Agrupación
Con los paréntesis `()` se pueden agrupar varios caracteres, lo cual es muy útil para:
- poder aplicarle a todo ese grupo un mismo cuantificador;
- guardar ese grupo, es decir, poder usarlo después (en la misma regex de búsqueda o en la de reemplazo).

Por ejemplo, podemos querer buscar palabras repetidas. Una forma de hacerlo es sabiendo la palabra que se repite:

In [6]:
a = re.search(r"(for ??){2}", "Always look for for the bright side of life")
print(a)

<re.Match object; span=(12, 19), match='for for'>


Observa que en el anterior ejemplo hemos usado `?` una vez como cuantificador y otra como indicador de _laziness_. Esto nos permite capturar el espacio después de `for` solo cuando nos conviene (el que está entre las dos palabras, pero no el de después de la última palabra).

Pero supongamos que el texto es demasiado largo o no podemos leerlo a priori, vamos, que no tenemos forma de saber qué palabras se van a repetir. Para ello podemos agrupar la expresión regular con la que describimos la palabra y después `\1`:

In [8]:
a = re.findall(r"(\w+) \1", "Always look for for the bright side of of life")
print(a)

['for', 'of']


Pero, desde luego, donde más útiles son los grupos es en las sustituciones. A veces necesitamos describir en el patrón una parte del texto que no queremos sustituir, así que podemos captuarla y usar en el reemplazo el mismo grupo. Por ejemplo, para separar los guiones de las palabras:

In [10]:
a = re.sub(r"([^ ])[-—]([^ ])", r"\1 - \2", "de ninguna manera-dijo")
print(a)

de ninguna manera - dijo


Por supuesto, también podemos cambiar de sitio los grupos:

In [11]:
a = re.sub(r"(\w+)@(\w+)\.\w+", r"Máquina: \2\nUsuario: \1", "user@machine.ext")
print(a)

Máquina: machine
Usuario: user


## Grupos anidados
Los grupos se pueden anidar, es decir, podemos hacer que un fragmento de string pertenezca a dos grupos distintos.

Imagina que en las siguientes strings queremos capturar tanto el año como el mes y el año. Tendremos que meter entre paréntesis una regex que capture el año, y luego poner otro paréntesis alrededor de la que incluya el mes:

In [50]:
a = re.findall("(.*(\d{4}))", """Jan 1987\nMay 1969\nAug 2011""")
print(a)

[('Jan 1987', '1987'), ('May 1969', '1969'), ('Aug 2011', '2011')]


## Ejercicios
### 090301
Encuentra cada oración por separado en esta cita de las _Meditaciones_ de Marco Aurelio, rellenando la celda con la regex correcta. El resultado debe ser una lista con las distintas oraciones.

In [ ]:
a = re.findall("", "No actúes en la idea de que vas a vivir diez mil años. La necesidad ineludible pende sobre ti. Mientras vives, mientras es posible, sé virtuoso.")
print(a)

### 090302
Encuentra todas las líneas aéreas:

In [ ]:
airlines = """Andes Líneas Aéreas
Plus Ultra Líneas Aéreas
Líneas Aéreas del Estado"""

a = re.findall("", airlines)
print(a)

### 090303
Captura por un lado el ancho y por otro el alto en estas medidas:

In [ ]:
a = re.findall(r"", "1280x720")
print(a)
a = re.findall(r"", "1920x1600")
print(a)
a = re.findall(r"", "1024x768")
print(a)

### 090304
Ahora, con los mismos textos que en el ejercicio 090206, no solo queremos capturar los errores, sino además sustituirlos por las palabras correctas (`lo`, `las` y `les`):

In [ ]:
texto = """todo estaba oscuro
era to más parecido
una de tas mejores obras
según decían, «tes asustaban»"""

resultados = re.sub(r"\bt([aoe]s?)\b", r"", texto)
print(resultados)

### 090305
La regex para encontrar las 3 formas de referirse a las regex (`regex`, `regexp` y `regular expressions`) es…

In [ ]:
frases = [
    "Las regex se estudian en el tema 9.",
    "También podemos encontrarlas escritas como regexp.",
    "La librería re quiere decir regular expressions."
]

for frase in frases:
    resultado = re.search("", frase)
    print(resultado)

### 090306
Captura en grupos el prefijo de estos números de teléfono estadounidenses, que están escritos con distintos formatos. Los prefijos son, en cada línea: `415`, `650`, `416`, `202`, `403` y `416`.

In [ ]:
numeros = """415-555-1234
650-555-2345
(416)555-3456
202 555 4567
4035555678
1 416 555 9292"""
a = re.findall("", numeros)
print(a)